In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
from time import time
from sklearn.datasets import load_digits

In [2]:
from flaml.data import load_openml_dataset
X_train, X_test, y_train, y_test = load_openml_dataset(dataset_id=1169, data_dir='./')

load dataset from ./openml_ds1169.pkl
Dataset name: airlines
X_train.shape: (404537, 7), y_train.shape: (404537,);
X_test.shape: (134846, 7), y_test.shape: (134846,)


In [1]:
y_train = pd.Series(y_train,dtype="category",name="Delay")
y_test = pd.Series(y_test,dtype="category",name="Delay")

NameError: name 'pd' is not defined

In [4]:
columns = ["Airline", "Flight", "AirportFrom", "AirportTo", "DayOfWeek", "Time", "Length"]
dtypes = {"Airline": int, "AirportFrom": int, "AirportTo": int, "DayOfWeek": int, "Time": float, "Length": float}
dtypes2 = {"Airline": "category", "AirportFrom": "category", "AirportTo": "category", "DayOfWeek": "category", "Time": float, "Length": float}

X_train = pd.DataFrame(X_train, columns=columns).drop("Flight", axis=1).astype(dtypes).astype(dtypes2)
X_test = pd.DataFrame(X_test, columns=columns).drop("Flight", axis=1).astype(dtypes).astype(dtypes2)

In [3]:
data.drop(["city", "enrollee_id"], axis=1, inplace=True)

KeyError: "['city' 'enrollee_id'] not found in axis"

In [4]:
data["last_new_job"] = data["last_new_job"].replace({">4": "5", "never": 0}).astype(float)

In [5]:
data["experience"] = data["experience"].replace({">20": "21", "<1": "0"}).astype(float)

In [6]:
ordinal_columns={
    "enrolled_university": ["no_enrollment", "Part time course", "Full time course"],
    "education_level": ["Primary School", "High School", "Graduate", "Masters", "Phd"],
    "company_size": ["<10", "10-49", "50-99", "100-500", "500-999", "1000-4999", "5000-9999", "10000+"],
}

In [3]:
data = pd.read_csv("datasets/income.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income >50K
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [4]:
data["education"] = data["education"].replace(
    {
        'Preschool': 'dropout',
        '10th': 'dropout',
        '11th': 'dropout',
        '12th': 'dropout',
        '1st-4th': 'dropout',
        '5th-6th': 'dropout',
        '7th-8th': 'dropout',
        '9th': 'dropout',
        'HS-Grad': 'HighGrad',
        'HS-grad': 'HighGrad',
        'Some-college': 'CommunityCollege',
        'Assoc-acdm': 'CommunityCollege',
        'Assoc-voc': 'CommunityCollege',
        'Bachelors': 'Bachelors',
        'Masters': 'Masters',
        'Prof-school': 'Masters',
        'Doctorate': 'Doctorate'
    }
)

In [5]:
data["marital-status"] = data["marital-status"].replace(
    {
        'Never-married': 'NotMarried',
        'Married-AF-spouse': 'Married',
        'Married-civ-spouse': 'Married',
        'NotMarried': 'NotMarried',
        'Separated': 'Separated',
        'Divorced': 'Separated',
        'Widowed': 'Widowed',
    }
)

In [6]:
ordinal_columns={
    "education": ["dropout", "HighGrad", "CommunityCollege", "Bachelors", "Masters", "Doctorate"],
}

In [7]:
data["native-country"] = data["native-country"].replace(
{"Canada": "North America", "Cuba": "South America", "Dominican-Republic": "South America", "El-Salvador": "South America", "Guatemala": "South America",
                   "Haiti": "South America", "Honduras": "South America", "Jamaica": "South America", "Mexico": "South America", "Nicaragua": "South America",
                   "Outlying-US(Guam-USVI-etc)": "North America", "Puerto-Rico": "North America", "Trinadad&Tobago": "South America",
                   "United-States": "North America", "Cambodia": "Asia", "China": "Asia", "Hong": "Asia", "India": "Asia", "Iran": "Asia", "Japan": "Asia", "Laos": "Asia",
          "Philippines": "Asia", "Taiwan": "Asia", "Thailand": "Asia", "Vietnam": "Asia", "Columbia": "South America", "Ecuador": "South America", "Peru": "South America", "England": "West Europe", "France": "West Europe", "Germany": "West Europe", "Greece": "West Europe", "Holand-Netherlands": "West Europe",
            "Ireland": "West Europe", "Italy": "West Europe", "Portugal": "West Europe", "Scotland": "West Europe", "Hungary": "East Europe", "Poland": "East Europe", "Yugoslavia": "East Europe", "South": "?"})


In [8]:
data["capital"] = data["capital-gain"]-data["capital-loss"]
data.drop(["capital-gain", "capital-loss"], axis=1, inplace=True)
data["education-num"] = data["education-num"].astype(float)

In [9]:
from automl.components import ComponentLevel

In [10]:
from sklearn.model_selection import RepeatedStratifiedKFold
from automl.search.automl import AutoML

am = AutoML(
    random_state=42,
    level="common",
    #target_metric="f1",
    trainer_config={
        #"secondary_level": ComponentLevel.UNCOMMON,
        "cache": True,
        "early_stopping": True,
        "return_test_scores_during_tuning": True,
        "tuning_time": 60*15,
        "stacking_level": 0,
        "tune_kwargs": {"max_concurrent": 1, "trainable_n_jobs": 4}
    },
    #cv=RepeatedStratifiedKFold(random_state=420, n_repeats=2, n_splits=4)
    cv=4,
)

In [11]:
t_s = time()
am.fit(data.drop("income >50K", axis=1), data["income >50K"], ordinal_columns={"education":ordinal_columns})
#am.fit(data.drop("Purchase", axis=1), data["Purchase"])
#am.fit(data.drop("default", axis=1), data["default"], ordinal_columns={"EDUCATION": list(sorted(data["EDUCATION"].unique()))})
#am.fit(data.drop("quality", axis=1), data["quality"]>=6)
#am.fit(data.drop("target", axis=1), data["target"])
#am.fit(data.drop("target", axis=1), data["target"], ordinal_columns=ordinal_columns)
#am.fit(X_train, y_train, X_test=X_test, y_test=y_test)
t_end = time() - t_s

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best validation score',
         …

suggest 0ec7a21a, 12
0ec7a21a final suggestion by 0: {'Estimator': 'LGBMClassifier(boosting_type=gbdt, num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=200, subsample_for_bin=200000, class_weight=balanced, min_split_gain=0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0009765625, reg_lambda=1.0, random_state=0, n_jobs=1, silent=True)', 'FeatureSelector': 'Passthrough(None)', 'Estimator__LGBMClassifier__n_estimators': 200, 'Estimator__LGBMClassifier__num_leaves': 31, 'Estimator__LGBMClassifier__learning_rate': 0.1, 'Estimator__LGBMClassifier__subsample': 1.0, 'Estimator__LGBMClassifier__colsample_bytree': 1.0, 'Estimator__LGBMClassifier__reg_alpha': 0.0009765625, 'Estimator__LGBMClassifier__reg_lambda': 1.0, 'Estimator__LGBMClassifier__min_child_samples': 20, 'Estimator__LGBMClassifier__class_weight': 'balanced', 'Imputer__CombinedSimpleImputer__categorical_strategy': 'most_frequent', 'ColumnScaling__ScalerN

(pid=17141) Index(['age', 'workclass', 'education-num', 'marital-status', 'occupation',
(pid=17141)        'relationship', 'race', 'sex', 'hours-per-week', 'native-country',
(pid=17141)        'capital'],
(pid=17141)       dtype='object')
(pid=17135) fitting on train took 0.556530237197876
(pid=17135) scoring took 0.38760948181152344
Trial SklearnTrainable_0ec7a21a reported mean_validation_score=0.825965844401262,estimator_fit_time=3.1897506713867188,metrics={'accuracy': 0.8354217854217854, 'balanced_accuracy': 0.8395368442877305, 'roc_auc': 0.925517806323111, 'precision': 0.6149300294644551, 'recall': 0.8474754551718716, 'specificity': 0.8315982334035894, 'f1': 0.7126684045253961, 'matthews_corrcoef': 0.6166442438015277, 'optimized_precision': 0.825965844401262},test_metrics={'accuracy': 0.8355238914138312, 'balanced_accuracy': 0.8477835301731111, 'roc_auc': 0.9308016832359864, 'precision': 0.6110912343470483, 'recall': 0.8714285714285714, 'specificity': 0.8241384889176508, 'f1': 0.71

suggest 127c2066, 11
127c2066 final suggestion by 0: {'ColumnScaling__ScalerNormalizer': 'CombinedScalerTransformer(sparse_threshold=0.3, n_jobs=1, transformer_weights=None, verbose=False)', 'Estimator': 'LogisticRegression(penalty=elasticnet, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=balanced, random_state=0, solver=saga, max_iter=200, multi_class=auto, verbose=0, warm_start=False, n_jobs=None, l1_ratio=0)', 'FeatureSelector': 'SHAPSelectFromModelClassification(estimator=LGBMClassifier, threshold=mean, prefit=False, norm_order=1, max_features=None, importance_getter=auto, n_estimators=auto, random_state=0)', 'Estimator__LogisticRegression__C': 1.0, 'Estimator__LogisticRegression__l1_ratio': 0, 'Estimator__LogisticRegression__class_weight': 'balanced', 'Imputer__CombinedSimpleImputer__categorical_strategy': 'most_frequent', 'Imputer': 'CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, cate

suggest 1e7b734e, 9
1e7b734e final suggestion by 0: {'ColumnScaling__ScalerNormalizer': 'MinMaxScaler(feature_range=(0, 1), copy=True, clip=False)', 'Estimator': 'KNeighborsClassifier(n_neighbors=5, weights=distance, p=2, metric=precomputed, metric_params=None, n_jobs=None)', 'FeatureSelector': 'Passthrough(None)', 'Estimator__KNeighborsClassifier__n_neighbors': 5, 'Estimator__KNeighborsClassifier__weights': 'distance', 'Imputer__CombinedSimpleImputer__categorical_strategy': 'most_frequent', 'KNNTransformer__KNNTransformer__p': 2, 'Imputer': 'CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, categorical_fill_value=missing_value, verbose=0, copy=True, n_jobs=1, transformer_weights=None)', 'Imbalance': 'Passthrough(None)', 'ColumnOrdinal__OrdinalEncoder': 'passthrough', 'ColumnEncoding__BinaryEncoder': "BinaryEncoder(categories=auto, handle_unknown=use_encoded_value, dtype=<class 'bool'>, unknown_value=None)", 'ColumnEncoding__Catego

...

suggest 224e1e66, 0
DEADLINE: 4.882123947143555
best global score=-0.8352110238054636
estimator LGBMClassifier(boosting_type=gbdt, num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=200, subsample_for_bin=200000, class_weight=balanced, min_split_gain=0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0009765625, reg_lambda=1.0, random_state=0, n_jobs=1, silent=True)
best_score=-0.8352110238054636 best_score_old=-0.831612558057016
best_cost=118.62196040153503 best_cost_old=29.72327208518982
total_time=304.9091351032257 best_eval_time=3.3138585090637207
estimator RandomForestClassifier(n_estimators=100, randomization_type=rf, criterion=gini, max_depth=10, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=1e-10, max_features=0.2, random_state=0, max_leaf_nodes=None, min_impurity_decrease=1e-10, bootstrap=True, oob_score=False, n_jobs=None, class_weight=balanced, ccp_alpha=0.0, verbose=0, warm_start=

suggest 26d63e82, 0
DEADLINE: -2.7208504676818848
best global score=-0.8352110238054636
estimator LGBMClassifier(boosting_type=gbdt, num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=200, subsample_for_bin=200000, class_weight=balanced, min_split_gain=0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0009765625, reg_lambda=1.0, random_state=0, n_jobs=1, silent=True)
best_score=-0.8352110238054636 best_score_old=-0.831612558057016
best_cost=118.62196040153503 best_cost_old=29.72327208518982
total_time=307.45266795158386 best_eval_time=3.3138585090637207
estimator RandomForestClassifier(n_estimators=100, randomization_type=rf, criterion=gini, max_depth=10, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=1e-10, max_features=0.2, random_state=0, max_leaf_nodes=None, min_impurity_decrease=1e-10, bootstrap=True, oob_score=False, n_jobs=None, class_weight=balanced, ccp_alpha=0.0, verbose=0, warm_sta

Trial name,status,loc,ColumnEncoding__BinaryEncoder,ColumnEncoding__CategoricalEncoder,ColumnOrdinal__OrdinalEncoder,ColumnScaling__ScalerNormalizer,Estimator,Estimator__CatBoostClassifierBinary__auto_class_weights,Estimator__CatBoostClassifierBinary__max_depth,Estimator__CatBoostClassifierBinary__n_estimators,Estimator__KNeighborsClassifier__n_neighbors,Estimator__KNeighborsClassifier__weights,Estimator__LGBMClassifier__class_weight,Estimator__LGBMClassifier__colsample_bytree,Estimator__LGBMClassifier__learning_rate,Estimator__LGBMClassifier__min_child_samples,Estimator__LGBMClassifier__n_estimators,Estimator__LGBMClassifier__num_leaves,Estimator__LGBMClassifier__reg_alpha,Estimator__LGBMClassifier__reg_lambda,Estimator__LGBMClassifier__subsample,Estimator__LogisticRegression__C,Estimator__LogisticRegression__class_weight,Estimator__LogisticRegression__l1_ratio,Estimator__RandomForestClassifier__class_weight,Estimator__RandomForestClassifier__criterion,Estimator__RandomForestClassifier__max_depth,Estimator__RandomForestClassifier__max_features,Estimator__RandomForestClassifier__min_impurity_decrease,Estimator__RandomForestClassifier__min_samples_leaf,Estimator__RandomForestClassifier__min_samples_split,Estimator__RandomForestClassifier__min_weight_fraction_leaf,Estimator__RandomForestClassifier__n_estimators,Estimator__RandomForestClassifier__randomization_type,FeatureSelector,Imbalance,Imputer,Imputer__CombinedSimpleImputer__categorical_strategy,KNNTransformer,KNNTransformer__KNNTransformer__p,SVMKernelApproximation,iter,total time (s),mean_validation_score,estimator_fit_time
SklearnTrainable_0ec7a21a,TERMINATED,,passthrough,passthrough,"OrdinalEncoder(categories=auto, handle_unknown=use_encoded_value, dtype=<class 'numpy.uint16'>, unknown_value=None)",passthrough,"LGBMClassifier(boosting_type=gbdt, num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=200, subsample_for_bin=200000, class_weight=balanced, min_split_gain=0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0009765625, reg_lambda=1.0, random_state=0, n_jobs=1, silent=True)",,,,,,balanced,1,0.1,20,200,31,0.000976562,1,1,,,,,,,,,,,,,,Passthrough(None),Passthrough(None),"CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, categorical_fill_value=missing_value, verbose=0, copy=True, n_jobs=1, transformer_weights=None)",most_frequent,passthrough,,Passthrough(None),1,4.36813,0.825966,3.18975
SklearnTrainable_127c2066,TERMINATED,,"BinaryEncoder(categories=auto, handle_unknown=use_encoded_value, dtype=<class 'bool'>, unknown_value=None)","CatBoostEncoderBinary(base_transformer=CatBoostEncoder(cols=[], random_state=0), cv=5, return_same_type=True, is_classification=True, random_state=None)",passthrough,"CombinedScalerTransformer(sparse_threshold=0.3, n_jobs=1, transformer_weights=None, verbose=False)","LogisticRegression(penalty=elasticnet, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=balanced, random_state=0, solver=saga, max_iter=200, multi_class=auto, verbose=0, warm_start=False, n_jobs=None, l1_ratio=0)",,,,,,,,,,,,,,,1,balanced,0,,,,,,,,,,,"SHAPSelectFromModelClassification(estimator=LGBMClassifier, threshold=mean, prefit=False, norm_order=1, max_features=None, importance_getter=auto, n_estimators=auto, random_state=0)",Passthrough(None),"CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, categorical_fill_value=missing_value, verbose=0, copy=True, n_jobs=1, transformer_weights=None)",most_frequent,passthrough,,Passthrough(None),1,2.04758,0.755987,1.13164
SklearnTrainable_13c2bc8c,TERMINATED,,passthrough,passthrough,"OrdinalEncoder(categories=auto, handle_unknown=use_encoded_value, dtype=<class 'numpy.uint16'>, unknown_value=None)",passthrough,"CatBoostClassifierBinary(n_estimators=200, learning_rate=None, max_depth=6, task_type=CPU, verbose=False, rand

Finished loading model, total used 200 iterations
Finished loading model, total used 66 iterations
Finished loading model, total used 66 iterations
Finished loading model, total used 66 iterations
Finished loading model, total used 66 iterations
Finished loading model, total used 193 iterations
Finished loading model, total used 102 iterations
Finished loading model, total used 390 iterations
Finished loading model, total used 271 iterations
Finished loading model, total used 66 iterations
Finished loading model, total used 148 iterations
Finished loading model, total used 176 iterations
Finished loading model, total used 227 iterations
Finished loading model, total used 7 iterations
Finished loading model, total used 313 iterations
Finished loading model, total used 99 iterations
Finished loading model, total used 377 iterations
Finished loading model, total used 261 iterations
Finished loading model, total used 142 iterations
Finished loading model, total used 66 iterations
Finished 

,Pipeline,Test accuracy,Test balanced_accuracy,Test roc_auc,Test precision,Test recall,Test specificity,Test f1,Test matthews_corrcoef,Test optimized_precision,...,Validation precision,Validation recall,Validation specificity,Validation f1,Validation matthews_corrcoef,Validation optimized_precision,Stacking Level,Total Time (s),Estimator Fit Time (s),Dataset Fraction
Id,,,,,,,,,,,,,,,,,,,,,
c280b756,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.845474,0.844232,0.925070,0.635104,0.841837,0.846627,0.724002,0.631461,0.842637,...,0.628604,0.795272,0.850909,0.702164,0.600306,0.803713,0,36.444157,23.476756,1.0
9a113fe8,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844000,0.843261,0.925018,0.632184,0.841837,0.844685,0.722101,0.628912,0.842310,...,0.632297,0.803775,0.851664,0.707773,0.608099,0.811203,0,4.893002,3.563299,1.0
d7d27efe,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844368,0.843330,0.926954,0.633013,0.841327,0.845332,0.722453,0.629356,0.841993,...,0.628479,0.807516,0.848536,0.706810,0.606856,0.813887,0,5.277553,3.515446,1.0
ead11a7a,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844368,0.845594,0.928961,0.631699,0.847959,0.843229,0.724025,0.631855,0.841571,...,0.627358,0.819079,0.845515,0.710422,0.612119,0.823267,0,5.881199,3.942861,1.0
e190cda6,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.842280,0.842651,0.926525,0.628517,0.843367,0.841935,0.720261,0.626537,0.841430,...,0.624217,0.810407,0.845191,0.705183,0.604721,0.815804,0,4.804056,3.037159,1.0
dc1e0e0c,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.843140,0.842172,0.925370,0.630793,0.840306,0.844038,0.720630,0.626854,0.840924,...,0.628830,0.797823,0.850531,0.703270,0.601862,0.805861,0,7.404252,4.968781,1.0
66e82522,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844245,0.842726,0.926617,0.633077,0.839796,0.845656,0.721930,0.628570,0.840768,...,0.636467,0.797654,0.855440,0.707973,0.608348,0.806567,0,6.101295,4.174871,1.0
927413b8,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844000,0.841867,0.925786,0.632999,0.837755,0.845980,0.721124,0.627380,0.839115,...,0.631783,0.798844,0.852258,0.705522,0.604979,0.807044,0,5.215822,3.836806,1.0
e32a6088,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844982,0.847915,0.927916,0.631798,0.853571,0.842259,0.726128,0.635034,0.838311,...,0.625104,0.813977,0.845083,0.707108,0.607423,0.818843,0,8.920648,5.693457,1.0


In [23]:
a = am.get_pipeline_by_id("91a28710")

Finished loading model, total used 66 iterations


In [24]:
a.steps.pop(-1)

('Estimator',
 LogisticRegression(class_weight='balanced', l1_ratio=0.5, max_iter=200,
                    penalty='elasticnet', random_state=42, solver='saga'))

In [25]:
x = a.fit_transform(am.X_, am.y_)

In [26]:
x

,marital-status,occupation,relationship,age,education-num,capital
19359,-0.124056,-0.473291,-0.008957,-0.473684,0.000000,0.0
15003,0.879363,0.151460,0.895744,-0.105263,0.666667,0.0
12490,-0.121258,-0.715864,-0.173605,-0.684211,0.000000,0.0
32292,0.873204,-0.174355,0.886522,0.210526,0.000000,0.0
5163,-0.121809,0.151460,-0.247393,-0.526316,1.333333,0.0
...,...,...,...,...,...,...
31450,0.884189,0.535585,0.898115,0.947368,1.333333,0.0
20560,-0.001838,0.497922,-0.113345,0.105263,0.000000,0.0
23650,0.884189,1.175944,0.898115,0.947368,-0.333333,0.0
25301,0.879363,1.208270,0.895744,-0.578947,1.000000,0.0


In [10]:
x.dtypes

marital-status    float32
occupation        float32
relationship      float32
age               float32
education-num     float32
hours-per-week    float32
capital           float32
dtype: object

In [27]:
am.X_test_

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,capital
17581,49,Self-emp-not-inc,Some-college,10,Divorced,Exec-managerial,Not-in-family,Amer-Indian-Eskimo,Female,40,United-States,0
16280,27,Private,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,40,United-States,0
24719,33,Private,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,41,United-States,0
3986,30,Private,HS-grad,9,Separated,Adm-clerical,Not-in-family,White,Female,40,United-States,0
5138,41,Federal-gov,Some-college,10,Never-married,Protective-serv,Own-child,White,Male,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6837,23,State-gov,12th,8,Never-married,Exec-managerial,Not-in-family,White,Male,40,United-States,0
7133,21,Private,7th-8th,4,Never-married,Farming-fishing,Unmarried,White,Male,40,United-States,0
25363,24,Private,Some-college,10,Never-married,Sales,Other-relative,White,Male,35,United-States,0
13085,44,State-gov,HS-grad,9,Married-civ-spouse,Protective-serv,Husband,White,Male,38,United-States,0


In [13]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

[CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42)]

In [14]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

[CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42)]

In [ ]:
a.score(am.X_test_, am.y_test_)

In [12]:
am.visualize_pipeline("65a71b94")

Finished loading model, total used 313 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 313 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 91 iterations


In [11]:
am.results_.sort_values("Test accuracy", ascending=False)[:60]

,Pipeline,Test accuracy,Test balanced_accuracy,Test roc_auc,Test precision,Test recall,Test specificity,Test f1,Test matthews_corrcoef,Test optimized_precision,...,Validation precision,Validation recall,Validation specificity,Validation f1,Validation matthews_corrcoef,Validation optimized_precision,Stacking Level,Total Time (s),Estimator Fit Time (s),Dataset Fraction
Id,,,,,,,,,,,,,,,,,,,,,
4de38d26,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.874831,0.809212,0.929950,0.771182,0.682653,0.935771,0.724222,0.645655,0.718433,...,0.772099,0.653461,0.938778,0.707789,0.628628,0.690873,1,7.030053,5.999565,1.0
3803c584,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.873603,0.805442,0.919485,0.772063,0.673980,0.936903,0.719695,0.641040,0.710386,...,0.574575,0.880123,0.793193,0.695221,0.596715,0.762177,1,7.377254,5.904064,1.0
65a71b94,"LogisticRegression w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasSHAPSelectFromModel, PandasOrdinalEncoder, KFoldCatBoostEncoder, PandasStandardScaler, PandasQuantileTransformer",0.871760,0.826701,0.932272,0.730847,0.739796,0.913606,0.735294,0.650703,0.766637,...,0.732808,0.720456,0.916662,0.726576,0.640848,0.749562,1,35.281595,19.560393,1.0
e50551d0,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.870655,0.799842,0.930195,0.767868,0.663265,0.936418,0.711744,0.631740,0.699900,...,0.768591,0.641558,0.938724,0.699332,0.619113,0.679112,1,60.473898,49.453166,1.0
118d399a,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.859722,0.768079,0.915299,0.772667,0.591327,0.944831,0.669942,0.591272,0.629600,...,0.768893,0.593946,0.943363,0.670176,0.590405,0.631923,0,6.619399,5.768160,1.0
57b742d4,"RandomForestExtraTreesClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder, KFoldCatBoostEncoder",0.854809,0.842366,0.905205,0.660082,0.818367,0.866365,0.730752,0.639763,0.826319,...,0.665415,0.791526,0.872537,0.721929,0.628525,0.804347,1,21.710557,12.963434,1.0
77528d16,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.845105,0.823084,0.917290,0.648030,0.780612,0.865556,0.708169,0.608823,0.793504,...,0.642489,0.737630,0.869788,0.686774,0.580699,0.755743,0,17.626041,11.079301,1.0
1_VotingSoft,PandasVotingClassifier,0.845105,0.850783,0.932678,0.630459,0.861735,0.839832,0.728174,0.638358,0.832233,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0
9a39f0f8,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844982,0.847915,0.927916,0.631798,0.853571,0.842259,0.726128,0.635034,0.838311,...,0.625104,0.813977,0.845083,0.707108,0.607423,0.818843,0,17.043041,10.464772,1.0


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

lr = LogisticRegression(random_state=42).fit(X_train, y_train)
dt = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

/home/baum/.local/share/virtualenvs/automl-KTV6y-vU/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
predictions = [lr.predict_proba(X_train), dt.predict_proba(X_train)]

In [3]:
from automl_models.components.estimators.ensemble import PandasStackingClassifier
stack = PandasStackingClassifier(estimators=[("lr", lr), ("dt", dt)], final_estimator=LogisticRegression(random_state=42)).fit(X_train, y_train)

fitting estimator LogisticRegression(random_state=42)
fitting estimator DecisionTreeClassifier(random_state=42)
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
doing cv for LogisticRegression(random_state=42).predict_proba
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED 

In [4]:
p = stack.predict_proba(X_test)

In [5]:
stack._saved_test_predictions=predictions
pp = stack.predict_proba(X_test)

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [6]:
p == pp

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [6]:
am.results_.loc["0_VotingUniform"]

Pipeline                          PandasVotingClassifier
Test accuracy                                   0.832944
Test balanced_accuracy                          0.848349
Test roc_auc                                         NaN
Test precision                                  0.605559
Test recall                                     0.878061
Test specificity                                0.818638
Test f1                                         0.716785
Test matthews_corrcoef                          0.624876
Test optimized_precision                        0.797921
Validation accuracy                                  NaN
Validation balanced_accuracy                         NaN
Validation roc_auc                                   NaN
Validation precision                                 NaN
Validation recall                                    NaN
Validation specificity                               NaN
Validation f1                                        NaN
Validation matthews_corrcoef   

In [7]:
p = am.get_pipeline_by_id("0_VotingUniform")

Finished loading model, total used 200 iterations
Finished loading model, total used 200 iterations


In [8]:
p.score(am.X_test_, am.y_test_)

0.8329443557302543

In [16]:
p._final_estimator

RandomForestClassifier(class_weight='balanced_subsample', max_depth=7,
                       max_features=0.9041555101461939,
                       min_impurity_decrease=3.384914384121488e-07,
                       min_samples_leaf=20, min_samples_split=19,
                       min_weight_fraction_leaf=1.7645447745993082e-06,
                       n_estimators=221, random_state=42)

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, clone, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from deslib.des import KNORAE, METADES, KNORAU
import numpy as np

In [2]:
rfc = RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True)
X, y = load_breast_cancer(return_X_y=True)


In [234]:
data = pd.read_csv("datasets/wine.csv")

In [246]:
X = data.drop("type", axis=1).to_numpy()
y = LabelEncoder().fit_transform(data["type"].to_numpy())

In [5]:
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

In [15]:
class DESCV(ClassifierMixin, BaseEstimator):
    def __init__(self, des, classifier_pool, cv):
        self.des = des
        self.classifier_pool = classifier_pool
        self.cv = cv
    def fit(self, X, y):
        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        transformed_y = self.le_.transform(y)

        des = []
        for train_ind, test_ind in self.cv.split(X, y):
            rfc_ = clone(self.classifier_pool).fit(X[train_ind], y[train_ind])
            des_ = clone(self.des)
            des_.set_params(pool_classifiers=rfc_)
            des_.fit(X[test_ind], y[test_ind])
            des.append(des_)
        self.estimators_ = des

    def _predict(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([est.predict(X) for est in self.estimators_]).T

    def _collect_probas(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([clf.predict_proba(X) for clf in self.estimators_])

    def predict(self, X):
        maj = np.argmax(self.predict_proba(X), axis=1)
        maj = self.le_.inverse_transform(maj)

        return maj

    def predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting."""
        #check_is_fitted(self)
        avg = np.average(self._collect_probas(X), axis=0)
        return avg

In [7]:
X, y = make_classification(n_samples=7000, random_state=42)

In [16]:
descv = DESCV(KNORAE(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [17]:
desucv = DESCV(KNORAU(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [18]:
metadescv = DESCV(METADES(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [11]:
%%time
np.mean(cross_val_score(rfc, X, y, scoring="roc_auc_ovr_weighted"))

CPU times: user 10.4 s, sys: 16.8 ms, total: 10.5 s
Wall time: 10.5 s


0.991752412179687

In [261]:
np.mean(cross_val_score(rfc, X, y, scoring="f1", error_score="raise"))

0.9792307757731944

In [19]:
%%time
np.mean(cross_val_score(descv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

CPU times: user 48.5 s, sys: 8.19 s, total: 56.7 s
Wall time: 41.7 s


0.9916842502509222

In [ ]:
np.mean(cross_val_score(descv, X, y, scoring="f1"))

In [ ]:
%%time
np.mean(cross_val_score(desucv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(desucv, X, y, scoring="f1", error_score="raise"))

In [ ]:
%%time
np.mean(cross_val_score(metadescv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(metadescv, X, y, scoring="f1", error_score="raise"))

In [46]:
rfc.fit(X, y)
rfc.oob_decision_function_

array([[0.90697674, 0.09302326],
       [0.96551724, 0.03448276],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ]])

In [49]:
np.argmax(rfc.oob_decision_function_, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [47]:
rfc.oob_decision_function_.shape

(569, 2)